In [ ]:
from statsmodels import datasets
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import dautil as dl
import numpy as np
import seaborn as sns
import warnings
from IPython.display import HTML

In [ ]:
warnings.filterwarnings(action='ignore',
                        message='.*Mean of empty slice.*')
warnings.filterwarnings(action='ignore',
                        message='.*Degrees of freedom <= 0 for slice.*')

In [ ]:
context = dl.nb.Context('discrete_wavelet')
lr = dl.nb.LatexRenderer(chapter=6, start=12, context=context)
lr.render(r'Y_t=X_{t}-m \text{  for } t=1,2, \dots ,n')
lr.render(r'Z_t= \sum_{i=1}^{t} Y_{i}  \text{  for }  t=1,2, \dots ,n')
lr.render(r'R_t = \max\left (Z_1, Z_2, \dots, Z_t  \right )- \min\left (Z_1, Z_2, \dots, Z_t  \right ) \text{  for } t=1,2, \dots, n')
lr.render(r'S_{t}= \sqrt{\frac{1}{t} \sum_{i=1}^{t}\left ( X_{i} - m(t) \right )^{2}}  \text{  for } t=1,2, \dots ,n')
lr.render(r'\left ( R/S \right )_{t} = \frac{R_{t}}{S_{t}} \text{  for } t=1,2, \dots, n')

In [ ]:
def calc_rescaled_range(X):
    N = len(X)

    # 1. Mean
    mean = X.mean()

    # 2. Y mean adjusted
    Y = X - mean

    # 3. Z cumulative deviates
    Z = np.array([Y[:i].sum() for i in range(N)])

    # 4. Range R
    R = np.array([0] + [np.ptp(Z[:i]) for i in range(1, N)])

    # 5. Standard deviation S
    S = np.array([X[:i].std() for i in range(N)])

    # 6. Average partial R/S
    return [np.nanmean(R[:i]/S[:i]) for i in range(N)]

In [ ]:
data = datasets.get_rdataset('Nile', cache=True).data
cA, cD = pywt.dwt(data['Nile'].values, 'haar')
coeff = pd.DataFrame({'cA': cA, 'cD': cD})

In [ ]:
%matplotlib inline
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 2, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 2, context)
sp.ax.plot(data['time'], data['Nile'])
sp.label()

cp = dl.plotting.CyclePlotter(sp.next_ax())
cp.plot(range(len(cA)), cA, label='Approximation coefficients')
cp.plot(range(len(cD)), cD, label='Detail coefficients')
sp.label()

sp.next_ax().loglog(range(len(cA)), calc_rescaled_range(cA), 
                    label='Approximation coefficients')
sp.ax.loglog(range(len(cD)), calc_rescaled_range(cD), 
             label='Detail coefficients')
sp.label()

range_df = pd.DataFrame(data={'Year': data.index,
                              'Rescaled range':
                              calc_rescaled_range(data['Nile'])})
sp.next_ax().set(xscale="log", yscale="log")
sns.regplot('Year', 'Rescaled range', range_df, ax=sp.ax, order=1,
            scatter_kws={"s": 100})
sp.label()
HTML(sp.exit())